In [1]:
import glob
from pathlib import Path
from tensorboard.backend.event_processing import event_accumulator as ea
import pandas as pd
import numpy as np
import re

try:
    from tensorboard.errors import DataNotFoundError         
except Exception:                                             
    class DataNotFoundError(Exception):                       
        pass

LIE

In [ ]:
# REGULAR
# lie_results = pd.read_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/lie_test_results.csv")
# anthropic_results = pd.read_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/anthropic_bias_results.csv")
# reward_bench_results = pd.read_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/reward_bench_results.csv")
# rm_bench_results = pd.read_csv("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/rm_bench_results.csv")

In [3]:
paths = glob.glob("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/*/*/*/*/*/*/*/runs/*/events.*")
len(paths)

160

In [4]:

def get_tb(paths, label="eval/loss"):
    records = []
    for ev_path in paths:
        acc = ea.EventAccumulator(ev_path, size_guidance={"scalars": 0})
        try:
            acc.Reload()
            events = acc.Scalars(label)          
            if not events:                             
                continue
        except (DataNotFoundError, KeyError, Exception):
            continue

        latest = events[-1] 

        path = str(Path(ev_path).parent.parent)
        splits = path.split("fold_")[1].split("/")
        fold = int(splits[0])
        lr = float(splits[1].split("_")[1])
        epochs = int(splits[2].split("_")[1])                         
        records.append(
            dict(
                run=path, 
                fold=fold,
                lr=lr,
                epochs=epochs,
                eval_loss=latest.value,
            )
        )
    return records

In [6]:
records = get_tb(paths)
df_latest = pd.DataFrame(records).sort_values("eval_loss").reset_index(drop=True)

In [7]:
df_latest = df_latest.drop_duplicates(["lr", "epochs", "fold"])

In [8]:
median_df = (
    df_latest.groupby(["lr", "epochs"], as_index=False)
      .agg(
          median_eval_loss=("eval_loss", "median"),
          std_eval_loss=("eval_loss", "std"),

          fold_count=("fold", "count")
      )
      .sort_values(("median_eval_loss"))
)
best_lr = median_df["lr"].iloc[0]
best_epochs = median_df["epochs"].iloc[0]

In [9]:
df_latest[(df_latest["lr"] == best_lr) & (df_latest["epochs"] == best_epochs)]["run"].tolist() 

['/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/fold_4/lr_2e-06_cosine/epochs_3/batch_32/ws_0.1/score_scale_1.0/1/runs',
 '/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/fold_0/lr_2e-06_cosine/epochs_3/batch_32/ws_0.1/score_scale_1.0/1/runs',
 '/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/fold_1/lr_2e-06_cosine/epochs_3/batch_32/ws_0.1/score_scale_1.0/1/runs',
 '/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/fold_3/lr_2e-06_cosine/epochs_3/batch_32/ws_0.1/score_scale_1.0/1/runs',
 '/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_regular_scale/fold_2/lr_2e-06_cosine/epochs_3/batch_32/ws_0.1/score_scale_1.0/1/runs']

In [10]:
def parse_eval_tag(tag: str):
    m = re.match(r"lr_([0-9.eE+-]+)-epochs_(\d+)-fold_(\d+)", tag)
    if not m:
        return None, None, None  
    lr_str, epochs_str, fold_str = m.groups()
    try:
        lr = float(lr_str)  
    except ValueError:
        lr = None
    return lr, int(epochs_str), int(fold_str)

lie_results[["lr", "epochs", "fold"]] = lie_results["eval_results_tag"].apply(
    lambda x: pd.Series(parse_eval_tag(x))
)

reward_bench_results[["lr", "epochs", "fold"]] = reward_bench_results["eval_results_tag"].apply(
    lambda x: pd.Series(parse_eval_tag(x))
)

rm_bench_results[["lr", "epochs", "fold"]] = rm_bench_results["eval_results_tag"].apply(
    lambda x: pd.Series(parse_eval_tag(x))
)


In [11]:
lie_results = lie_results.drop_duplicates()

In [12]:
lie_results = lie_results.merge(df_latest)

In [13]:
print(0.000010)

1e-05


In [14]:
lie_results.groupby(["lr", "epochs"])["fold"].apply(set)  

lr        epochs
0.000001  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          3.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          5.0       {0.0, 1.0, 2.0, 3.0, 4.0}
0.000002  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          3.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          5.0       {0.0, 1.0, 2.0, 3.0, 4.0}
0.000005  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          3.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          5.0       {0.0, 1.0, 2.0, 3.0, 4.0}
0.000010  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          3.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          5.0       {0.0, 1.0, 2.0, 3.0, 4.0}
0.000020  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          3.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          5.0       {0.0, 1.0, 2.0, 3.0, 4.0}
0.000030  1.0       {0.0, 1.0, 2.0, 3.0, 4.0}
          2.0    

In [15]:
median_df = (
    lie_results.groupby(["lr", "epochs"], as_index=False)
      .agg(
          median_eval_loss=("eval_loss", "median"),
          median_ratio=("ratio", "median"),
          std_ratio=("ratio", "std"),
          median_length_weight=("length_weight", "median"),
          median_correctness_weight=("correctness_weight", "median"),
          fold_count=("fold", "count")
      )
      .sort_values(["lr", "epochs"])
)
median_df.sort_values("median_eval_loss")


,lr,epochs,median_eval_loss,median_ratio,std_ratio,median_length_weight,median_correctness_weight,fold_count
6,0.000002,3.0,0.614311,-0.016479,0.013136,0.627958,-0.010348,5
5,0.000002,2.0,0.617115,-0.031004,0.027650,0.468944,-0.014539,5
7,0.000002,5.0,0.620854,-0.003849,0.032602,0.795630,-0.003254,5
10,0.000005,3.0,0.622655,0.160634,0.032709,1.003993,0.171844,5
9,0.000005,2.0,0.623825,0.098361,0.025086,0.951690,0.090091,5
21,0.000030,2.0,0.627292,-0.059674,0.035306,0.970483,-0.051100,5
26,0.000050,3.0,0.631469,-0.101759,0.054637,0.700322,-0.075439,5
27,0.000050,5.0,0.631619,-0.082562,0.053354,0.889349,-0.087286,5
24,0.000050,1.0,0.633313,-0.076389,0.019435,0.771484,-0.045068,5
8,0.000005,1.0,0.634652,0.022767,0.035435,1.006739,0.018409,5


In [33]:
ratio = median_df.sort_values("median_eval_loss")["median_ratio"].iloc[0]
std_ratio = median_df.sort_values("median_eval_loss")["std_ratio"].iloc[0]
best_lr = median_df.sort_values("median_eval_loss")["lr"].iloc[0]
best_epochs = median_df.sort_values("median_eval_loss")["epochs"].iloc[0]
print(f"{ratio:.2f} ± {std_ratio:.2f}")


-0.02 ± 0.01


In [53]:
median_df = (
    reward_bench_results.groupby(["lr", "epochs"], as_index=False)
      .agg(
          median_chat=("Chat", "median"),
          std_chat=("Chat", "std"),
          median_chat_hard=("Chat Hard", "median"),
          std_chat_hard=("Chat Hard", "std"),
          median_safety=("Safety", "median"),
          std_safety=("Safety", "std"),
          median_reasoning=("Reasoning", "median"),
          std_reasoning=("Reasoning", "std")
      )
      .sort_values(["lr", "epochs"])
)
median_df[(median_df["lr"] == best_lr) & (median_df["epochs"] == best_epochs)]
row = median_df.loc[
    (median_df["lr"] == best_lr) & (median_df["epochs"] == best_epochs)
].iloc[0]

print(
    f"Chat:          {row.median_chat:.2f} ± {row.std_chat:.2f}\n"
    f"Chat (Hard):   {row.median_chat_hard:.2f} ± {row.std_chat_hard:.2f}\n"
    f"Safety:        {row.median_safety:.2f} ± {row.std_safety:.2f}\n"
    f"Reasoning:     {row.median_reasoning:.2f} ± {row.std_reasoning:.2f}"
)



Chat:          0.68 ± 0.05
Chat (Hard):   0.45 ± 0.01
Safety:        0.54 ± 0.02
Reasoning:     0.78 ± 0.02


In [54]:
rm_bench_results.columns

Index(['eval_results_tag', 'hard_acc', 'normal_acc', 'easy_acc', 'lr',
       'epochs', 'fold'],
      dtype='object')

In [58]:
median_df = (
    rm_bench_results.groupby(["lr", "epochs"], as_index=False)
      .agg(
          median_hard=("hard_acc", "median"),
          std_hard=("hard_acc", "std"),
          median_normal=("normal_acc", "median"),
          std_normal=("normal_acc", "std"),
          median_easy=("easy_acc", "median"),
          std_easy=("easy_acc", "std"),

      )
      .sort_values(["lr", "epochs"])
)
median_df[(median_df["lr"] == best_lr) & (median_df["epochs"] == best_epochs)]
row = median_df.loc[
    (median_df["lr"] == best_lr) & (median_df["epochs"] == best_epochs)
].iloc[0]

print(
    f"Hard:          {row.median_hard:.2f} ± {row.std_hard:.2f}\n"
    f"Normal:   {row.median_normal:.2f} ± {row.std_normal:.2f}\n"
    f"Easy:        {row.median_easy:.2f} ± {row.std_easy:.2f}\n"
)



Hard:          0.52 ± 0.03
Normal:   0.49 ± 0.00
Easy:        0.45 ± 0.03



TRUE

In [5]:
paths = glob.glob("/mnt/efs/shivamsinghal/reliability-aware-preference-learning/data/reward_models/Llama-3.1-8B/llama_true/*/*/*/batch_32/*/*/*/*/runs/*/events.*")
len(paths)

332

In [10]:
eval_loss_records = get_tb(paths)
calib_loss_records = get_tb(paths, "eval/calibration_val_loss")

In [23]:
df_eval_loss = pd.DataFrame(eval_loss_records).sort_values("eval_loss").reset_index(drop=True)
df_calib_loss = pd.DataFrame(calib_loss_records).sort_values("eval_loss").reset_index(drop=True)
df_calib_loss["calibration_loss"] = df_calib_loss["eval_loss"]
df_calib_loss.drop("eval_loss", inplace=True, axis=1)
df_true = df_eval_loss.merge(df_calib_loss)

In [25]:
df_true = df_true.drop_duplicates(["fold", "lr", "epochs"])

In [31]:
median_df = (
    df_true.groupby(["lr", "epochs"], as_index=False)
      .agg(
          median_eval_loss=("eval_loss", "median"),
          std_eval_loss=("eval_loss", "std"),
          median_calib_loss=("calibration_loss", "median"),
          std_calib_loss=("calibration_loss", "std"),

      )
      .sort_values(["lr", "epochs"])
)
median_df.sort_values("median_calib_loss")


,lr,epochs,median_eval_loss,std_eval_loss,median_calib_loss,std_calib_loss
19,0.000020,5,0.629384,0.070207,0.540747,0.027890
20,0.000030,1,0.548888,0.049320,0.547780,0.666978
23,0.000030,5,0.636069,0.067502,0.561807,0.147365
12,0.000010,1,0.558130,0.037365,0.562262,0.532365
27,0.000050,5,0.623831,0.099484,0.564184,0.297495
26,0.000050,3,0.617005,0.084274,0.568551,0.574035
10,0.000005,3,0.570023,0.049773,0.568638,0.247797
13,0.000010,2,0.550497,0.043395,0.576699,0.082743
15,0.000010,5,0.663291,0.092509,0.580826,0.086059
24,0.000050,1,0.589483,0.033284,0.584641,0.304514


- Check tensorboard for true models to make sure it is fine
- Check Cassidy true model train
- Ensemble or not?
- Schedule TRUE p and beta